In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms, models

from PIL import Image, ImageFile
import pickle

import pandas as pd
import numpy as np
import random
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os

In [2]:
def set_seed(seed):
    os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
#Labels (parecerFinal):
# Branco = 0
# PPI = 1
table = "tabelaAlunosPPI_cropped.csv" # 201257335.jpg foi removida
df = pd.read_csv(table, index_col=0) 

In [5]:
# Separa o dataframe em dois: Um para brancos e outro para PPI
# Isso será importante para organizar os dados em train e test
df_branco = df[(df['parecerFinal'] == 0)]
df_PPI = df[df['parecerFinal'] == 1]


df_branco = df_branco.reset_index(drop=True)
df_PPI = df_PPI.reset_index(drop=True)

In [6]:
len(df_branco)

452

In [7]:
len(df_PPI)

4691

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.data['caminho'][index]
        label = self.data['parecerFinal'][index]
        image = Image.open(image_path)
        image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return self.data.shape[0]

In [9]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-10,10)),
    transforms.ToTensor(),
    transforms.Normalize(mean=119.03655493743558, std=58.19396798203402)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [10]:
num_epochs = 15
lr = 0.1
batch_size = 256

In [11]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_data_branco, test_data_branco = train_test_split(df_branco, test_size=0.1, random_state=42)
train_data_PPI, test_data_PPI = train_test_split(df_PPI, test_size=0.1, random_state=42)

train_data = pd.concat([train_data_branco, train_data_PPI], ignore_index=True)
test_data = pd.concat([test_data_branco, test_data_PPI], ignore_index=True)

In [12]:
custom_train = CustomDataset(train_data, transform=train_transform)
custom_test = CustomDataset(test_data, transform=test_transform)

In [13]:
train_loader = DataLoader(custom_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(custom_test, batch_size=batch_size, shuffle=True)

In [14]:
len(custom_train) + len(custom_test)

5143

In [15]:
weights = pickle.load(open('resnet50_scratch_weight.pkl', 'rb'))
model = models.resnet50(weights=weights)

c:\Users\DELL\anaconda3\envs\CUDA\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
for param in model.parameters():
    param.requires_grad = False

# for param in model.layer4.parameters():
#      param.requires_grad = True

model.fc = nn.Linear(2048, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
# Treino

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.2)

model.train()
for epoch in range(num_epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output.squeeze(), labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f'Epoch: {epoch + 1} Loss: {running_loss/len(train_loader)}')

Epoch: 1 Loss: 12.410825180379968
Epoch: 2 Loss: 6.340959166225634
Epoch: 3 Loss: 4.329094077411451
Epoch: 4 Loss: 2.902830487803409
Epoch: 5 Loss: 3.6560117633719194
Epoch: 6 Loss: 5.649132734850833
Epoch: 7 Loss: 4.024655665221967
Epoch: 8 Loss: 1.4077760357605784
Epoch: 9 Loss: 0.6234535010237443
Epoch: 10 Loss: 0.48596695692915665
Epoch: 11 Loss: 0.4636328063513103
Epoch: 12 Loss: 0.4251170017217335
Epoch: 13 Loss: 0.41403331254657944
Epoch: 14 Loss: 0.39360418915748596
Epoch: 15 Loss: 0.3880019878086291


In [18]:
# Teste

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

Accuracy: 91 %


In [19]:
# Matriz de confusão

y_pred = []
y_true = []
classes = ['branco', 'PPI']

model.eval()
model.cpu()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted = predicted.cpu()
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)

    cm = confusion_matrix(y_true, y_pred)

    print('\t\tPredicted Labels\n')

    print(f"True Labels Branco {cm[0]}")
    print(f"\t       PPI {cm[1]}") 
    print('\t\t   Branco PPI')

		Predicted Labels

True Labels Branco [ 0 46]
	       PPI [  0 470]
		   Branco PPI
